In [1]:
import tensorflow as tf
import numpy as np
import numpy.random as rng
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\Users\prash\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def sigma(x):
    return tf.div(tf.constant(1.0,dtype=tf.float64),tf.add(tf.constant(1.0,dtype=tf.float64),tf.exp(tf.negative(x))))

def sigmaprime(x):
    return tf.multiply(sigma(x),tf.subtract(tf.constant(1.0,dtype=tf.float64),sigma(x)))

In [33]:
batch_size=50
inshape=[batch_size,784]
hiddenshape=20
outshape=[batch_size,10]

#Training Parameters 
iterations=100
eta=tf.constant(0.000001,dtype=tf.float64)

#Training Data Inputs
x=tf.placeholder(tf.float64,inshape,name='x')
y=tf.placeholder(tf.float64,outshape,name='y')

#Feedback alignment
B=tf.Variable(rng.randn(hiddenshape,10))
w_1fa=tf.Variable(rng.randn(hiddenshape,784))
w_2fa=tf.Variable(rng.randn(hiddenshape,10))

hidoutfa=sigma(tf.matmul(x,tf.transpose(w_1fa)))
#y_pred_fa=(tf.matmul(hidoutfa,w_2fa))
y_pred_fa=sigma(tf.matmul(hidoutfa,w_2fa))

e_fa=y_pred_fa-y
cost_fa=tf.reduce_sum(tf.pow(e_fa,2))/2
#Be=tf.matmul(B,tf.transpose(e_fa))

h_primefa = hidoutfa*(1-hidoutfa)

Be=tf.matmul(e_fa,tf.transpose(B))

dLw1fa = tf.transpose(tf.matmul(tf.transpose(x),tf.multiply(Be, h_primefa)))

#dLw1fa=tf.matmul(tf.transpose(Be),x)
#dLw1fa=tf.matmul(Be,x)
dLw2fa=tf.matmul(tf.transpose(hidoutfa),e_fa)


#Back Propogation
w1bp=tf.Variable(w_1fa.initialized_value())
w2bp=tf.Variable(w_2fa.initialized_value())

hidoutbp=tf.matmul(x,tf.transpose(w1bp))
y_pred_bp=sigma(tf.matmul(hidoutbp,w2bp))


e_bp=y_pred_bp-y
cost_bp=tf.reduce_sum(tf.pow(e_bp,2))/2
We=tf.matmul(e_bp,tf.transpose(w2bp))

h_primebp = hidoutbp*(1-hidoutbp)

dLw1bp=tf.transpose(tf.matmul(tf.transpose(x),tf.multiply(We, h_primebp)))

dLw2bp=tf.matmul(tf.transpose(hidoutfa),e_bp)


In [34]:
def updatefa(w_1,w_2):    
    new_w1 = w_1.assign(w_1-tf.multiply(eta,dLw1fa))
    new_w2 = w_2.assign(w_2-tf.multiply(eta,dLw2fa)) 
    return new_w1, new_w2 

def updatebp(w1bp,w2bp):
    new_w1bp=w1bp.assign(w1bp-tf.multiply(eta,dLw1bp))
    new_w2bp=w2bp.assign(w2bp-tf.multiply(eta,dLw2bp))
    return new_w1bp,new_w2bp


In [35]:
init = tf.global_variables_initializer()
costfa=[]
costbp=[]
with tf.Session() as sess:
    sess.run(init)
    for idx in range(iterations):
        (train_x, train_y) = mnist.train.next_batch(batch_size)
    
        sess.run(updatefa(w_1fa,w_2fa), feed_dict={x: train_x, y: train_y})
        cfa = sess.run(cost_fa, feed_dict={x: train_x, y: train_y})
        costfa.append(cfa)

        sess.run(updatebp(w1bp,w2bp),feed_dict={x: train_x, y: train_y})
        cbp = sess.run(cost_bp, feed_dict={x: train_x, y: train_y})
        costbp.append(cbp)
        
        if idx % 50==0:
            print("iteration: %d Feedbackalignment:%f\tBackprop:%f"%(idx,cfa,cbp))
            #delta_fa=sess.run(Be)
            #delta_bp=sess.run(We)
            #storedot.append(np.dot(delta_fa,delta_bp))

iteration: 0 Feedbackalignment:79.149242	Backprop:118.025018
iteration: 50 Feedbackalignment:85.701819	Backprop:nan


In [39]:
S=tf.Session()
dLw1



InvalidArgumentError: You must feed a value for placeholder tensor 'y_8' with dtype double and shape [50,10]
	 [[Node: y_8 = Placeholder[dtype=DT_DOUBLE, shape=[50,10], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'y_8', defined at:
  File "C:\Users\prash\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\prash\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\prash\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\prash\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\prash\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\prash\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "C:\Use

In [17]:
dLw1bp

<tf.Tensor 'MatMul_32:0' shape=(784, 20) dtype=float64>

In [7]:
tf.multiply(Be,h_prime)

<tf.Tensor 'Mul_1:0' shape=(50, 20) dtype=float64>